In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install hf_xet
!pip install huggingface_hub[hf_xet]

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd


In [7]:
E5_model = "intfloat/multilingual-e5-base"
model = SentenceTransformer(E5_model)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [39]:
texts = df["full_text"].tolist()

embeddings = model.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True 
)

print(embeddings.shape)  


Batches: 100%|██████████| 41/41 [05:24<00:00,  7.92s/it]

(2604, 768)


In [40]:
np.save("game_embeddings.npy", embeddings)
df.to_csv("merged_with_embeddings_meta.csv", index=False)


In [ ]:

def build_filter_mask(df, selected_genres=None, selected_categories=None):
    mask = pd.Series(True, index=df.index)

    if selected_genres:
        mask &= df["genres_list"].apply(
            lambda g: all(genre in g for genre in selected_genres)
        )

    if selected_categories:
        mask &= df["categories_list"].apply(
            lambda c: all(cat in c for cat in selected_categories)
        )

    return mask


In [ ]:
def boost_title_match(df_results, query, boost=0.03):
    q = query.lower().strip()
    df_results = df_results.copy()
    
    
    title_match = df_results["name"].str.lower().str.contains(q).astype(float)
    df_results["final_score"] = df_results["similarity"] + title_match * boost
    
    return df_results.sort_values("final_score", ascending=False)

In [ ]:
def recommend_games(query, selected_genres=None, selected_categories=None, top_k=10):
    mask = build_filter_mask(df, selected_genres, selected_categories)
    df_sub = df[mask]
    emb_sub = embeddings[mask.values]

    if df_sub.empty:
        return df_sub

    query_text = "query: " + query
    q_vec = model.encode(
        [query_text],
        convert_to_numpy=True,
        normalize_embeddings=True
    )[0]

    sims = emb_sub @ q_vec

    df_sub = df_sub.copy()
    df_sub["similarity"] = sims

    

    df_sub = boost_title_match(df_sub, query, boost=0.03)
    return df_sub.head(top_k)


In [ ]:
results = recommend_games(query="Sex")
results[["name", "similarity"]].head(10)


,name,similarity
539,The Elder Scrolls V: Skyrim,0.850610
1672,The Elder Scrolls V: Skyrim Special Edition,0.825012
390,Dungeon Siege III,0.831974
1544,Karma. Incarnation 1,0.827640
1126,Mount & Blade II: Bannerlord,0.826389
1313,Wars and Warriors: Joan of Arc,0.824434
1134,Rollers of the Realm,0.823756
393,FINAL FANTASY XIV Online,0.823265
1834,Kynseed,0.822328
1962,Grounded,0.822311
